In [37]:
from pipelines.test import *
import pandas as pd

In [38]:
stations_raw = 'C:\\Users\\dhruv\\IdeaProjects\\capstone\\data\\raw\\noaa_ghcnd\\ghcnd-stations.txt'
stations = read_station_metadata(stations_raw)

In [39]:
stations

,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
ID,,,,,,,,
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196
AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194
AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217
...,...,...,...,...,...,...,...,...
ZI000067969,-21.0500,29.3670,861.0,NaN,WEST NICHOLSON,NaN,NaN,67969
ZI000067975,-20.0670,30.8670,1095.0,NaN,MASVINGO,NaN,NaN,67975
ZI000067977,-21.0170,31.5830,430.0,NaN,BUFFALO RANGE,NaN,NaN,67977


In [40]:
dly = "C:\\Users\\dhruv\\Downloads\\ghcnd_all\\AGM00060550.dly"
data = read_ghcn_data_file(dly)

In [41]:
data

ELEMENT,PRCP,SNWD,TAVG,TMAX,TMIN
1973-01-05,NaN,NaN,35.0,70.0,-30.0
1973-02-18,NaN,NaN,4.0,50.0,-60.0
1973-03-02,NaN,NaN,65.0,NaN,0.0
1973-03-05,NaN,NaN,93.0,160.0,0.0
1973-04-25,NaN,NaN,178.0,NaN,110.0
...,...,...,...,...,...
2025-02-01,0.0,NaN,31.0,NaN,10.0
2025-02-02,0.0,NaN,19.0,61.0,NaN
2025-02-03,30.0,NaN,36.0,102.0,-30.0
2025-02-04,20.0,NaN,45.0,73.0,NaN


In [42]:
from pathlib import Path

raw_dir = Path('C:/Users/dhruv/Downloads/ghcnd_all\\')
raw_dir

WindowsPath('C:/Users/dhruv/Downloads/ghcnd_all')

In [45]:
import requests
from bs4 import BeautifulSoup


def list_files_http(url):
    """Lists files from an HTTP server directory."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        files = [a['href'] for a in soup.find_all('a') if a.has_attr('href')]
        return files
    except requests.exceptions.RequestException as e:
        print(f"Error accessing {url}: {e}")
        return []


def download_large_file(url, destination):
    try:
        with requests.get(url, stream=True) as response:
            response.raise_for_status()
            with open(destination, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
        print("File downloaded successfully!")
    except requests.exceptions.RequestException as e:
        print("Error downloading the file:", e)


In [44]:
url = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/superghcnd"  # Replace with the actual URL
files = list_files_http(url)

superghcnd_url = f'{url}/{files[-1]}'
superghcnd_url

'https://www.ncei.noaa.gov/pub/data/ghcn/daily/superghcnd/superghcnd_full_20250212.csv.gz'

In [47]:
download_large_file(superghcnd_url, "C:\\Users\\dhruv\\Downloads\\superghcnd.csv.gz")

KeyboardInterrupt: 

In [48]:
import requests
from tqdm import tqdm
import time

def download_large_file(url, destination):
    try:
        # Send a GET request to the URL with streaming enabled
        with requests.get(url, stream=True) as response:
            response.raise_for_status()  # Check for HTTP errors

            # Get the total file size from the response headers
            total_size = int(response.headers.get('content-length', 0))
            total_size_mb = total_size / (1024 * 1024)
            print(f"File size: {total_size_mb:.2f} MB")

            # Open the destination file in write-binary mode
            with open(destination, 'wb') as f:
                # Initialize the progress bar
                with tqdm(total=total_size_mb, unit='MB', unit_scale=True, desc='Downloading', ncols=100) as pbar:
                    start_time = time.time()
                    # Write the file in chunks to avoid memory issues
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            # Update the progress bar
                            pbar.update(len(chunk) / (1024 * 1024))
                            # Update the progress bar description with elapsed time
                            elapsed_time = time.time() - start_time
                            pbar.set_postfix(elapsed=f"{elapsed_time:.2f}s")
        print("File downloaded successfully!")
    except requests.exceptions.RequestException as e:
        print("Error downloading the file:", e)

In [49]:
download_large_file(superghcnd_url, "C:\\Users\\dhruv\\Downloads\\superghcnd.csv.gz")

File size: 11948.70 MB


Downloading:   0%|                               | 23.3/11.9k [00:02<18:02, 11.0MB/s, elapsed=2.31s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Downloading:   4%|█                             | 434/11.9k [01:59<52:55, 3.63MB/s, elapsed=119.70s]


KeyboardInterrupt: 